In [85]:
from functools import lru_cache
from typing import Callable, List

import numpy as np
from numpy.polynomial import polynomial as P
import pandas as pd
import plotly.express as px
from math import *

In [86]:
x_label = "x"
y_label = "u(x)"
color_label = "N"


def plot_solution_with_color(df):
    fig = px.line(df, x=x_label, y=y_label, color=color_label)
    fig.show()

In [87]:
import scipy.integrate as integrate


@lru_cache(maxsize=None)
def create_jacobi_polynomial(k: int, n: int, is_special=False):
    if n == 0:
        poly = P.Polynomial([1])
    elif n == 1:
        poly = P.Polynomial([0, k + 1])
    else:
        main_coefficient = (n + k + 2) / ((n + 2 * k + 2) * (n + 2))
        prev_coefficient = 2 * n + 2 * k + 3
        prev_prev_coefficient = n + k + 1

        prev_polynomial = create_jacobi_polynomial(k, n - 1, False)
        prev_prev_polynomial = create_jacobi_polynomial(k, n - 2, False)

        prev_polynomial = prev_coefficient * prev_polynomial * P.Polynomial([0, 1])
        prev_prev_polynomial = prev_prev_coefficient * prev_prev_polynomial

        poly = main_coefficient * (prev_polynomial - prev_prev_polynomial)
    if is_special:
        additional = P.Polynomial([1, 0, -1]) ** k
        poly = additional * poly
    return poly


def form_coord_system(N: int):
    system = [P.Polynomial([1])]
    if N >= 2:
        system.append(P.Polynomial([0, 1]))
    for i in range(N - 2):
        system.append(create_jacobi_polynomial(1, i, True))
    return system


def find_dot_product(y: Callable, z: Callable, i: int):
    func = lambda x: y(x) * z(x)
    return integrate.quad(func, -1, 1)[0]


def find_bilinear_form(y, z, p: Callable, r: Callable, alfa_1: int, alfa_2: int, beta_1: int, beta_2: int):
    y_deriv = y.deriv()
    z_deriv = z.deriv()

    func = lambda x: p(x) * y_deriv(x) * z_deriv(x) + r(x) * y(x) * z(x)
    if alfa_2 == 0:
        q_l = 0
    else:
        q_l = (alfa_1 / alfa_2) * p(-1) * y(-1) * z(-1)
    if beta_2 == 0:
        q_r = 0
    else:
        q_r = (beta_1 / beta_2) * p(1) * y(1) * z(1)
    return integrate.quad(func, -1, 1)[0] + q_l + q_r


def get_alfa_coefficients(N: int, coord_system, p: Callable, r: Callable,
                          f: Callable, alfa_1: int, alfa_2: int, beta_1: int, beta_2: int):
    A = []
    d = []

    for i in range(N):
        current_row = []
        fi_i = coord_system[i]
        for j in range(N):
            fi_j = coord_system[j]
            current_row.append(find_bilinear_form(fi_i, fi_j, p, r, alfa_1, alfa_2, beta_1, beta_2))
        A.append(current_row)
        d.append(find_dot_product(f, fi_i, i))
    return np.linalg.solve(A, d)


def find_solution_value_in_point(alfa_coefficients: List[float], coord_system, x):
    current_sum = 0
    for i in range(len(alfa_coefficients)):
        current_sum += alfa_coefficients[i] * coord_system[i](x)
    return current_sum


def get_solution(N: int, p: Callable, r: Callable, f: Callable, alfa_1: int,
                 alfa_2: int, beta_1: int, beta_2: int):
    coord_system = form_coord_system(N)
    alfa_coefficients = get_alfa_coefficients(N, coord_system, p, r, f, alfa_1, alfa_2, beta_1, beta_2)

    from_point = -1
    to_point = 1
    number_of_points = 100
    h = (to_point - from_point) / number_of_points

    solution = []
    for i in range(number_of_points + 1):
        x_n = from_point + i * h
        solution.append([x_n, find_solution_value_in_point(alfa_coefficients, coord_system, x_n)])
    return solution


# -(p(x) * u')' + r(x) * u = f(x)  [p(x) >= 0, r(x) >= 0]
# alfa_1 * u(-1) - alfa_2 * u'(-1) = 0
# beta_1 * u(1) - beta_2 * u'(1) = 0
def solve_with_ritz_method(N_range, p: Callable, r: Callable, f: Callable,
                           alfa_1: int, alfa_2: int, beta_1: int, beta_2: int):
    df_data = []
    for N in N_range:
        solution = get_solution(N, p, r, f, alfa_1, alfa_2, beta_1, beta_2)

        for x, u in solution:
            df_data.append([x, u, N])

    df = pd.DataFrame(df_data, columns=[x_label, y_label, color_label])
    plot_solution_with_color(df)

In [96]:
def plot_jacobi_functions(system):
    data = []
    f = -1
    t = 1
    points_number = 100
    net = np.linspace(f, t, num=points_number)

    for i, coord_func in enumerate(system):
        for point in net:
            value = coord_func(point)
            data.append((point, value, i))
    df = pd.DataFrame(data, columns=[x_label, y_label, color_label])
    plot_solution_with_color(df)


coord_system_length = 5
plot_jacobi_functions(form_coord_system(coord_system_length))

In [94]:
N_range = [2 ** i for i in range(1, 6)]

In [95]:
# variant 2
df = solve_with_ritz_method(
    N_range,
    lambda x: (2 + x) / (3 + x),
    lambda x: 1 + sin(radians(x)),
    lambda x: 1 - x,
    0,
    1,
    1,
    1
)

/tmp/ipykernel_14585/813536132.py:54: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

/tmp/ipykernel_14585/813536132.py:54: IntegrationWarning:

The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.



In [97]:
# variant 1
solve_with_ritz_method(
    N_range,
    lambda x: 1 / (2 + x),
    lambda x: cos(radians(x)),
    lambda x: 1 + x,
    1,
    0,
    1,
    0
)

/tmp/ipykernel_14585/813536132.py:54: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.



In [98]:
# variant 15
solve_with_ritz_method(
    N_range,
    lambda x: (x + 4) / (x + 5),
    lambda x: exp(x / 4),
    lambda x: 2 - x,
    0,
    -1,
    1,
    0
)

/tmp/ipykernel_14585/813536132.py:54: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

